### Dataset Loading and and separates it into the training, validation, and testing sets
Note: Dataset source: https://huggingface.co/datasets/dair-ai/emotion

In [1]:
import nlp

from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion", "split")

train = dataset['train']
val = dataset['validation']
test = dataset['test']

2024-09-01 10:39:27.832727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

### Separates training data into 2 arrays: “tweets” and “labels”.

In [3]:
def get_tweet(data):
    tweets = [x['text'] for x in data]
    labels = [x['label'] for x in data]
    return tweets, labels

tweets, labels = get_tweet(train)

tweets[0], labels[0]

('i didnt feel humiliated', 0)

### Initialize a tokenizer and calibrate it onto training data
This will assign each word a number by how commonly they appear in the dataset

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)

print(tweets[0])
print(tokenizer.texts_to_sequences(tweets[0]))

i didnt feel humiliated
[[2], [], [669], [2], [669], [1726], [43], [], [1997], [1550], [1550], [8884], [], [6051], [966], [93], [2], [8884], [2], [7], [43], [1550], [669]]


### Making all Sequences Same Shape

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen=50
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

padded_train_seq = get_sequences(tokenizer, tweets)

padded_train_seq


array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)

### Preparing Data for Model

In [14]:
import numpy as np

classes = set(labels)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(labels)

print(classes)
print(class_to_index)
print(index_to_class)
print(train_labels)

{0, 1, 2, 3, 4, 5}
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
[0 0 3 ... 1 3 0]


### Creating Model

In [16]:
from tensorflow import keras

model = keras.models.Sequential([
keras.layers.Embedding(10000,16),
keras.layers.Bidirectional(keras.layers.LSTM(20, return_sequences=True)),
keras.layers.Bidirectional(keras.layers.LSTM(20)),
keras.layers.Dense(6, activation='softmax')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

### Training Model

In [18]:
val_tweets, val_labels = get_tweet(val)
val_seq = get_sequences(tokenizer, val_tweets)
val_labels= names_to_ids(val_labels)
h = model.fit(
     padded_train_seq, train_labels,
     validation_data=(val_seq, val_labels),
     epochs=20,
     callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.3710 - loss: 1.5481 - val_accuracy: 0.6665 - val_loss: 0.9641
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 23s 46ms/step - accuracy: 0.7026 - loss: 0.8504 - val_accuracy: 0.7500 - val_loss: 0.7002
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 23s 45ms/step - accuracy: 0.8337 - loss: 0.4889 - val_accuracy: 0.8085 - val_loss: 0.5888
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 33s 66ms/step - accuracy: 0.9042 - loss: 0.3018 - val_accuracy: 0.8625 - val_loss: 0.4674
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.9455 - loss: 0.1878 - val_accuracy: 0.8765 - val_loss: 0.4091
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - accuracy: 0.9572 - loss: 0.1507 - val_accuracy: 0.8740 - val_loss: 0.4221
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 38s 77ms/step - accuracy: 0.9673 - loss: 0.1111 - val_accuracy: 0.8780 - val_loss: 0.4181
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.9730 - loss: 0.0906 - 

### Evaluating and Testing Model

In [19]:
test_tweets, test_labels=get_tweet(test)
test_seq = get_sequences(tokenizer, test_tweets)
test_labels=names_to_ids(test_labels)
model.evaluate(test_seq, test_labels)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8707 - loss: 0.4996


[0.5025031566619873, 0.8694999814033508]

In [20]:
import random

i = random.randint(0,len(test_labels)-1)
print('Sentence:', test_tweets[i])
print('Emotion:', index_to_class[test_labels[i]])
p = model.predict(np.expand_dims(test_seq[i], axis=0))[0]
print(test_seq[i])
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Predicted Emotion: ', pred_class)

Sentence: i feel like i can t truly get excited for this race because i have no idea whether or not i ll even be able to run it
Emotion: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[  2   3  14   2  40  43 395  55 268  16  23 947  37   2  21  96 486 556
  35  26   2 541  75  28 199   5 390  13   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Predicted Emotion:  1


In [21]:
sentence = 'i am not sure what to do'
sequence = tokenizer.texts_to_sequences([sentence])
paddedSequence = pad_sequences(sequence, truncating = 'post', padding='post', maxlen=maxlen)
p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Sentence:', sentence)
print('Predicted Emotion: ', pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Sentence: i am not sure what to do
Predicted Emotion:  1
